In [2]:
import random
import csv
import os
import json
from collections import defaultdict
from qiskit.circuit.random import random_circuit
from qiskit.qasm2 import dumps
from qiskit import transpile
from qiskit_aer import Aer
import math

# Configuration
MIN_QUBITS = 1            # Minimum number of qubits to generate    
MAX_QUBITS = 20                 # Maximum number of qubits to generate
BASE_SAMPLES = 3               # Base samples for max qubits (will propagate down)
MIN_DEPTH = 1                   # Minimum circuit depth
MAX_DEPTH = 15                  # Maximum circuit depth
TOTAL_SAMPLES = 100000          # Total number of samples to generate
IMAGE_DIR = 'Dataset/Source1/images/{qubit_number}-qubit-generated'    # Directory to store circuit images
CSV_FILENAME = 'Dataset/Source1/quantum_circuits_{qubit_number}_qubit.csv'  # Output CSV file



def calculate_distribution(num_qubits=20, total_data=100000):
    # Calculate the geometric progression ratio
    ratio = 4/3
    
    # Calculate the sum of the geometric series coefficients
    coefficients = [ratio**i for i in range(num_qubits)]
    total_coeff = sum(coefficients)
    
    # Calculate base value for first qubit
    base = total_data / total_coeff
    
    # Calculate exact values (before rounding)
    exact_values = [base * ratio**i for i in range(num_qubits)]
    
    # Round values and track the difference
    rounded = [math.ceil(v) for v in exact_values]
    current_total = sum(rounded)
    
    # Adjust to reach exactly 100000
    while current_total > total_data:
        max_index = rounded.index(max(rounded))
        rounded[max_index] -= 1
        current_total -= 1
        
    while current_total < total_data:
        min_index = rounded.index(min(rounded))
        rounded[min_index] += 1
        current_total += 1
    
    return rounded

def generate_circuit_data():
    """Main function to generate circuits and CSV data"""
    sample_distribution = calculate_distribution(MAX_QUBITS, TOTAL_SAMPLES)
    counter = defaultdict(int)

    # Generate circuits for each qubit count
    for qubits in range(MIN_QUBITS, MAX_QUBITS+1):
        csv_filename = CSV_FILENAME.format(qubit_number=qubits)
        with open(csv_filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['image_path1', 'image_path2', 'number_of_qubits', 'openqasm', 'number_of_shots', 'ground_truth'])
            
            # Create output directory if it doesn't exist
            image_dir = IMAGE_DIR.format(qubit_number=qubits)
            os.makedirs(image_dir, exist_ok=True)
            num_samples = sample_distribution[qubits-1]
            
            for _ in range(num_samples):
                # Generate random depth for each circuit
                depth = random.randint(MIN_DEPTH, MAX_DEPTH)
                
                # Create unique identifier for this qubit/depth combination
                key = (qubits, depth)
                serial_num = counter[key]
                counter[key] += 1
                
                # Generate random circuit
                circ = random_circuit(qubits, depth, measure=True)
                
                # Create filename with sorting-friendly format
                mpl_filename = f"q{qubits:02}_d{depth:03}_s{serial_num:04}_mpl.png"
                mpl_image_path = os.path.join(image_dir, mpl_filename)
                
                # Save circuit image
                circ.draw(output='mpl', filename=mpl_image_path)
                
                # Create filename with sorting-friendly format
                latex_filename = f"q{qubits:02}_d{depth:03}_s{serial_num:04}_latex.png"
                latex_image_path = os.path.join(image_dir, latex_filename)
                
                # Save circuit image
                circ.draw(output='latex', filename=latex_image_path)
                
                # Convert to OpenQASM
                qasm_str = dumps(circ)
                
                
                
                # Simulate circuit
                simulator = Aer.get_backend('aer_simulator')
                compiled = transpile(circ, simulator)
                result = simulator.run(compiled).result()
                counts = result.get_counts()
                
                # Calculate probabilities
                total_shots = sum(counts.values())
                probabilities = {state: count/total_shots for state, count in counts.items()}
                
                # Write to CSV
                writer.writerow([
                    mpl_image_path,
                    latex_image_path,
                    qubits,
                    qasm_str,
                    total_shots,
                    json.dumps(probabilities, indent=4)
                ])

if __name__ == '__main__':
    # result = calculate_distribution(MAX_QUBITS, TOTAL_SAMPLES)
    
    generate_circuit_data()
    print(f"Generated dataset with {BASE_SAMPLES} base samples in {CSV_FILENAME}")

KeyboardInterrupt: 

In [18]:
from qiskit import qpy

from qiskit.circuit.random import random_circuit
from qiskit import QuantumCircuit
from qiskit.circuit.library import UGate, CU3Gate, PhaseGate
import numpy as np
import qiskit.qasm2

# Step 1: Generate a random circuit
qubits = 10  # Number of qubits
depth = 10   # Depth of the circuit
random_circ = random_circuit(qubits, depth, measure=True)


def reverse_engineer_circuit2(circuit):
    """
    Uses Qiskit's built-in QASM2 conversion to generate non-deprecated Python code.
    """
    return qiskit.qasm2.dump(circuit)

def reverse_engineer_circuit(circuit):
    qiskit_code = []
    qiskit_code.append("from qiskit import QuantumCircuit")
    qiskit_code.append("from qiskit.circuit.library import UGate, CU3Gate, PhaseGate")
    qiskit_code.append(f"qc = QuantumCircuit({circuit.num_qubits}, {circuit.num_clbits})")

    for instruction, qargs, cargs in circuit.data:
        gate_name = instruction.name
        qubit_indices = [circuit.qubits.index(qubit) for qubit in qargs]

        if gate_name == "measure":
            clbit_indices = [circuit.clbits.index(clbit) for clbit in cargs]
            qiskit_code.append(f"qc.measure({qubit_indices[0]}, {clbit_indices[0]})")

        elif gate_name == "u1":  # Replace u1 with p (phase)
            qiskit_code.append(f"qc.p({instruction.params[0]}, {qubit_indices[0]})")

        elif gate_name == "u2":  # Replace u2 with u(π/2, φ, λ)
            qiskit_code.append(f"qc.u(np.pi/2, {instruction.params[0]}, {instruction.params[1]}, {qubit_indices[0]})")

        elif gate_name == "u3":  # Replace u3 with u
            qiskit_code.append(f"qc.u({instruction.params[0]}, {instruction.params[1]}, {instruction.params[2]}, {qubit_indices[0]})")

        elif gate_name == "cu":  # Replace cu with controlled U3
            qiskit_code.append(f"qc.append(CU3Gate({instruction.params[0]}, {instruction.params[1]}, {instruction.params[2]}), [{qubit_indices[1]}, {qubit_indices[0]}])")

        elif gate_name == "csdg":  # Controlled S-Dagger gate
            qiskit_code.append(f"qc.append(PhaseGate(-np.pi/2).control(), [{qubit_indices[0]}, {qubit_indices[1]}])")
            
        elif gate_name == "cu1":  # Replace cu1 with cp (controlled phase)
            qiskit_code.append(f"qc.cp({instruction.params[0]}, {qubit_indices[0]}, {qubit_indices[1]})")
            
        elif gate_name == "xx_minus_yy":
            qiskit_code.append(f"qc.rxx({instruction.params[0]}, {qubit_indices[0]}, {qubit_indices[1]})")
            qiskit_code.append(f"qc.ryy({instruction.params[1]}, {qubit_indices[0]}, {qubit_indices[1]})")


        elif instruction.params:  # Handle gates with parameters
            qiskit_code.append(f"qc.{gate_name}({', '.join(map(str, instruction.params))}, {', '.join(map(str, qubit_indices))})")

        else:  # Handle gates without parameters
            qiskit_code.append(f"qc.{gate_name}({', '.join(map(str, qubit_indices))})")

    qiskit_code.append("print(qc.draw())")
    return "\n".join(qiskit_code)

# Example usage:
random_circ = random_circuit(qubits, depth, measure=True)
qiskit_code = reverse_engineer_circuit2(random_circ)
print(qiskit_code)
random_circ.draw()


TypeError: dump() missing 1 required positional argument: 'filename_or_stream'

In [17]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import UGate, CU3Gate, PhaseGate
qc = QuantumCircuit(10, 10)
qc.rccx(9, 4, 0)
qc.rccx(3, 7, 5)
qc.cp(1.687512179756433, 2, 8)
qc.crx(3.544649782852167, 1, 6)
qc.c3sx(7, 9, 3, 1)
qc.ryy(2.810710304739744, 0, 4)
qc.x(8)
qc.cp(0.5559764472614454, 6, 5)
qc.ecr(5, 6)
qc.ecr(7, 3)
qc.rccx(4, 1, 2)
qc.rxx(0.15970912978922885, 8, 0)
qc.cs(2, 6)
qc.rcccx(4, 3, 5, 0)
qc.ccz(9, 7, 1)
qc.rccx(3, 4, 8)
qc.ry(1.0862738021756684, 5)
qc.ryy(5.530226785076235, 6, 2)
qc.rzz(0.9803202400261983, 7, 9)
qc.append(CU3Gate(2.323842417243001, 5.781827852397463, 6.23064474436693), [6, 3])
qc.ccz(5, 4, 2)
qc.c3sx(9, 0, 7, 8)
qc.ccz(3, 9, 7)
qc.cswap(4, 6, 1)
qc.cswap(8, 2, 5)
qc.rcccx(8, 6, 4, 9)
qc.ch(1, 7)
qc.cswap(0, 2, 5)
qc.c3sx(6, 9, 4, 7)
qc.cp(0.3363613978568575, 5, 2)
qc.c3sx(1, 8, 3, 0)
qc.c3sx(0, 4, 5, 6)
qc.ccz(8, 7, 9)
qc.measure(0, 0)
qc.measure(1, 1)
qc.measure(2, 2)
qc.measure(3, 3)
qc.measure(4, 4)
qc.measure(5, 5)
qc.measure(6, 6)
qc.measure(7, 7)
qc.measure(8, 8)
qc.measure(9, 9)
print(qc.draw())

AttributeError: 'QuantumCircuit' object has no attribute 'c3sx'

In [5]:
# read csv file using pandas with column name 'openqasm'
import pandas as pd

CSV_FILENAME = 'Dataset/Source1/quantum_circuits_1.csv' 

data = pd.read_csv(CSV_FILENAME)
test = data['openqasm'][:5]


print(test[0])


OPENQASM 2.0;
include "qelib1.inc";
gate r(param0,param1) q0 { u3(param0,param1 - pi/2,pi/2 - param1) q0; }
qreg q[1];
creg c[1];
u1(4.798817989861033) q[0];
u3(0.4130676302361043,1.6051317790859403,4.735618449222781) q[0];
sx q[0];
z q[0];
sx q[0];
x q[0];
u(0.8222490924899128,0.1102546292165667,0.33618110586817646) q[0];
h q[0];
s q[0];
s q[0];
u(4.413689674221526,1.789774756863345,2.933514450044832) q[0];
r(1.5188853853947208,0.4541066095710766) q[0];
h q[0];
s q[0];
sdg q[0];
measure q[0] -> c[0];


In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute, Aer
import numpy as np

backend = Aer.get_backend("qasm_simulator")

shots = 1024

def r(qc, param0, param1, a):
  qc.u(param0, param1 - np.pi / 2, np.pi / 2 - param1, a)

qc = QuantumCircuit()

q = QuantumRegister(1, 'q')
c = ClassicalRegister(1, 'c')

qc.add_register(q)
qc.add_register(c)

qc.p(4.798817989861033, q[0])
qc.u(0.4130676302361043, 1.6051317790859403, 4.735618449222781, q[0])
qc.sx(q[0])
qc.z(q[0])
qc.sx(q[0])
qc.x(q[0])
qc.u(0.8222490924899128, 0.1102546292165667, 0.33618110586817646, q[0])
qc.h(q[0])
qc.s(q[0])
qc.s(q[0])
qc.u(4.413689674221526, 1.789774756863345, 2.933514450044832, q[0])
r(qc, 1.5188853853947208, 0.4541066095710766, q[0])
qc.h(q[0])
qc.s(q[0])
qc.sdg(q[0])
qc.measure(q[0], c[0])

qc.draw()

job = execute(qc, backend=backend, shots=shots)
job_result = job.result()
print(job_result.get_counts(qc))

ImportError: cannot import name 'execute' from 'qiskit' (/home/cqilab/anaconda3/envs/anas_env/lib/python3.11/site-packages/qiskit/__init__.py)

In [1]:
!pip install cirq~=1.0.dev

INFO: pip is looking at multiple versions of cirq to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.5/566.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 2.2 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━